In [110]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
import numpy as np
import cv2
import mss
import time
import os
import warnings
import logging
from concurrent.futures import ThreadPoolExecutor
import pathlib

In [ ]:
# CONSTANTS

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
VID_OUTPUT_DIR = 'video_out'        # Directory to save video output
DETECT = True                       # Whether or not to label during capture process, False to simply record and output videos. No output if true
FPS = 15                            # Frames to capture per second
SECONDS_TO_CAPTURE = 60             # Video duration

In [112]:
if DEVICE == torch.device('cuda'):
    print(f'Using {DEVICE} {torch.cuda.get_device_name()} with {torch.cuda.mem_get_info()[0]/(1024**2)} MB of VRAM')

Using cuda NVIDIA GeForce RTX 3050 Ti Laptop GPU with 3144.4500007629395 MB of VRAM


In [113]:
if not os.path.exists('yolov5'):
  !git clone https://github.com/ultralytics/yolov5
  !pip install -r yolov5/requirements.txt
  

if not os.path.exists(VID_OUTPUT_DIR):
  os.makedirs(VID_OUTPUT_DIR)
  
warnings.simplefilter("ignore", FutureWarning)
logging.getLogger('ultralytics').setLevel(logging.ERROR)

In [114]:
pathlib.PosixPath = pathlib.WindowsPath # https://github.com/ultralytics/yolov5/issues/10240#issuecomment-1662573188
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./models/best.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\tiany/.cache\torch\hub\master.zip
YOLOv5  2024-10-30 Python-3.10.6 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [115]:
def detect(img, model):
  lanes = {
    0 : (10, 180),
    1 : (150, 320),
    2 : (300, 470),
    3 : (440, 610)
  }
  ret = []  
  res = model(img)
  
  for box in res.xyxy[0]:
    x_center = int((box[0] + box[2]) / 2)
    y_center = int((box[1] + box[3]) / 2)
    
    for lane, (start, end) in lanes.items():
      if start <= x_center <= end:
        break
    
    ret.append([lane, y_center])
  
  return ret 

def capture(region):
  with mss.mss() as sct:
    return sct.grab(region)

In [116]:
frames_to_capture = FPS * SECONDS_TO_CAPTURE
frames = []

monitor_1 = mss.mss().monitors[1]
print(f'Monitor: {monitor_1}')
t, l, w, h = monitor_1['top'], monitor_1['left'], monitor_1['width'], monitor_1['height']
region_1 = {'left': l+int(w * 0.338), 'top': t, 'width': w-int(w * 0.673), 'height': h} # Gameplay region
# region_2 = {'left': l+int(w * 0.67), 'top': t, 'width': w-int(w * 0.98), 'height': h-int(h * 0.33)} # Judgement counter region
# region_3 = {'left': l+int(l * 0.859), 'top': t, 'width': w-int(w * 0.820), 'height': h-int(h * 0.952)} # Score region

print(f'Capturing {region_1}')

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_1 = cv2.VideoWriter(f'{VID_OUTPUT_DIR}/output_1.mp4', fourcc, FPS, (region_1['width'], region_1['height']))

with ThreadPoolExecutor(max_workers=8) as executor:
  for i in range(frames_to_capture):
    time_start = time.time()
    thread_1 = executor.submit(capture, region_1)
    img_1 = thread_1.result()
    
    if DETECT:
      thread_1 = executor.submit(detect, np.array(img_1), model)
      notes = thread_1.result() # Outputs notes in [[lane_num, y_center_pos]...]
      print(notes)
    else:
      out_1.write(cv2.cvtColor(np.array(img_1), cv2.COLOR_BGRA2BGR))
    
    # If the current loop finishes faster than the FPS, wait.
    if time_start + 1/FPS > time.time():
      time.sleep(time_start + 1/FPS - time.time())

if not DETECT:
  out_1.release()
  print(f'Capture region saved to {VID_OUTPUT_DIR}/output_1.mp4')

Monitor: {'left': 1920, 'top': 0, 'width': 1920, 'height': 1080}
Capturing {'left': 2568, 'top': 0, 'width': 628, 'height': 1080}


KeyboardInterrupt: 